### Find out nearby Cites from your Custom City

In [10]:
from ipyleaflet import Map, Marker, CircleMarker, basemaps, LayersControl, Popup
from ipywidgets import HTML, Text, Button, VBox, IntSlider, Output
from geopy.geocoders import Nominatim
from geopy.distance import great_circle

In [13]:

big_cities = {
    "Tehran": (35.6892, 51.3890),
    "Mashhad": (36.2605, 59.6168),
    "Isfahan": (32.6546, 51.6680),
    "Karaj": (35.8327, 50.9916),
    "Shiraz": (29.5918, 52.5837),
    "Tabriz": (38.0962, 46.2738),
    "Qom": (34.6416, 50.8746),
    "Ahvaz": (31.3203, 48.6692),
    "Ardabil":(38.2432,48.2976),
    "Kermanshah": (34.3142, 47.0650),
    "Urmia": (37.5527, 45.0760),
    "Rasht": (37.2808, 49.5832),
    "Zahedan": (29.4963, 60.8629),
    "Hamadan": (34.7980, 48.5146),
    "Arak": (34.0917, 49.6892),
    "Yazd": (31.8974, 54.3569),
    "Kerman": (30.2839, 57.0834),
    "Sanandaj": (35.3149, 46.9988),
    "Bandar Abbas": (27.1832, 56.2666),
    "Khorramabad": (33.4878, 48.3558),
    "Abadan": (30.3473, 48.2934),
    "Zanjan": (36.6765, 48.4963),
    "Gorgan": (36.8456, 54.4436),
    "Qazvin": (36.2688, 50.0041),
    "Ilam": (33.6374, 46.4227),
    "Bojnurd": (37.4747, 57.3290),
    "Birjand": (32.8654, 59.2211),
    "Semnan": (35.5775, 53.3925),
    "Bushehr": (28.9234, 50.8203),
    "Yasuj": (30.6682, 51.5870),
    "Shahr-e Kord": (32.3256, 50.8644)
}

In [14]:

def geocode_city(city_name):
    geolocator = Nominatim(user_agent="geo_analysis")
    location = geolocator.geocode(city_name)
    if location is None:
        return None
    return (location.latitude, location.longitude)

def calculate_distance(point1, point2):
    return great_circle(point1, point2).kilometers

def find_nearby_cities(city_coords, cities, radius_km=200):
    return {
        city: coords
        for city, coords in cities.items()
        if calculate_distance(city_coords, coords) <= radius_km
    }

def show_map(city_name, city_coords, nearby_cities):
    print(f"Map center: {city_coords}")  
    m = Map(center=city_coords, zoom=6, basemap=basemaps.OpenStreetMap.Mapnik)
    marker = Marker(location=city_coords, title=city_name)
    m.add_layer(marker)
    for city, coords in nearby_cities.items():
        cm = CircleMarker(location=coords, radius=7, color='blue', fill_color='blue', fill_opacity=0.6)
        m.add_layer(cm)
    m.add_control(LayersControl())
    display(m)

In [15]:
# --- Interactive UI ---
city_input = Text(description="City:", placeholder="Enter city name")
radius_slider = IntSlider(value=200, min=50, max=1000, step=10, description="Radius (km):")
search_btn = Button(description="Find Nearby Cities")
output = Output()

def on_search_clicked(b):
    with output:
        output.clear_output()
        city_name = city_input.value.strip()
        radius = radius_slider.value
        if not city_name:
            print("Please enter a city name.")
            return
        city_coords = geocode_city(city_name)
        print(f"Coordinates of {city_name}: {city_coords}") 
        if city_coords is None:
            print(f"City '{city_name}' not found.")
            return
        nearby = find_nearby_cities(city_coords, big_cities, radius)
        if not nearby:
            print("No big cities found within the specified radius.")
        show_map(city_name, city_coords, nearby)

search_btn.on_click(on_search_clicked)

display(VBox([city_input, radius_slider, search_btn, output]))